In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.5 MB/s eta 0:00:00

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import transformers 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import datasets
torch.manual_seed(1)


In [4]:
from datasets import load_dataset
dataset = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})


In [6]:
test_range=[i for i in range(300)]
val_range=[i+300 for i in range(700)]

In [7]:
test_set=dataset["train"].select(test_range)
val_set=dataset["train"].select(val_range)
print(test_set)
print(val_set)

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 300
})
Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 700
})


In [8]:
test_set=test_set.remove_columns(["promptID", "pairID", "premise_binary_parse", "premise_parse",  "hypothesis_binary_parse", "hypothesis_parse", "genre"])
val_set=val_set.remove_columns(["promptID", "pairID", "premise_binary_parse", "premise_parse", "hypothesis_binary_parse", "hypothesis_parse", "genre"])

In [9]:
print(test_set)
print(val_set)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 300
})
Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 700
})


In [10]:
label_to_answers={"0":"Yes","1":"No"} #entail or not

In [11]:
def get_answer(example):
  return label_to_answers[str(example["label"])]

In [12]:
def preprocess(example):#change label 2 to 1, means not entail
  label=example["label"]
  if label==2:
    label=1
  return {"label": label}

In [13]:
test_set=test_set.map(preprocess)
val_set=val_set.map(preprocess)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [14]:
def get_complete_example(example):#get complete example using template or not
  answer=get_answer(example)
  return {"answer":answer}

In [15]:
test_set_complete=test_set.map(get_complete_example)
val_set_complete=val_set.map(get_complete_example)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [16]:
print(test_set_complete[0])
print(val_set_complete[0])

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'label': 1, 'answer': 'No'}
{'premise': "You're mad.", 'hypothesis': 'You are still sane.', 'label': 1, 'answer': 'No'}


In [17]:
foldername= "/content/drive/My Drive/IS/"
path=foldername+"multi_nli2/multi_nli2_dataset/"

In [ ]:
test_set_complete.save_to_disk(path+"test_data")
val_set_complete.save_to_disk(path+"fewshot_data")

Saving the dataset (0/1 shards):   0%|          | 0/300 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/700 [00:00<?, ? examples/s]